In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, LimitOrder, IB
util.startLoop()

In [2]:
# Set the root
from from_root import from_root
ROOT = from_root()

# Imports 
from utils import Vars, prepare_to_sow, place_orders, quick_pf
from loguru import logger

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET

In [3]:
# For new sows cancel existing open orders through API
df = prepare_to_sow(MARKET, 
                    build_from_scratch=False,
                    save_sow=False)

In [4]:
df[OPT_COLS]

,conId,symbol,undPrice,expiry,strike,right,dte,sdev,lot_size,margin,comm,ask,bid,last,optPrice,expPrice,prop,rom
0,675604050,BHEL,235.30,20240229,277.5,C,23.338569,1.554582,5250.0,235553.09,20.0,-1.0,-1.0,NaN,1.95,3.60,0.879954,1.253522
1,679103713,BHEL,235.30,20240328,292.5,C,51.338569,1.420733,5250.0,214683.92,20.0,-1.0,-1.0,NaN,3.80,7.20,0.844606,1.251156
2,675005097,BSOFT,834.15,20240229,950.0,C,23.338241,1.519962,1000.0,191154.26,20.0,-1.0,-1.0,NaN,7.80,15.30,0.871480,1.250156
3,669196229,RBLBANK,265.40,20240229,315.0,C,23.338449,1.614089,2500.0,141939.94,20.0,-1.0,-1.0,NaN,2.00,4.55,0.893492,1.251133
4,669153116,IEX,146.55,20240229,167.0,C,23.338551,1.483867,3750.0,123323.66,20.0,-1.0,-1.0,NaN,1.10,2.65,0.862156,1.257693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,674985804,AUROPHARM,1018.25,20240328,800.0,P,51.338006,-1.409418,1100.0,866703.49,20.0,-1.0,-1.0,NaN,0.85,138.55,-0.841289,1.250045
324,675121741,ZYDUSLIFE,760.90,20240328,640.0,P,51.338443,-1.459989,900.0,570392.18,20.0,-1.0,-1.0,NaN,0.50,111.45,-0.855707,1.250007
325,674978772,ALKEM,4924.05,20240328,4050.0,P,51.338552,-1.484533,200.0,800923.54,20.0,-1.0,-1.0,NaN,1.15,704.20,-0.862332,1.250036
326,675380684,TATAMOTOR,926.80,20240328,1070.0,C,51.338899,1.477877,1425.0,296049.51,20.0,NaN,NaN,NaN,NaN,36.55,0.860559,1.250309


In [5]:
# Make (contract, order) tuple
cos = [(contract , LimitOrder('Sell', qty, price))
    for contract, qty, price in zip(df.contract, df.lot_size, df.expPrice)]
len(cos)

328

In [ ]:
PORT

In [ ]:
from utils import get_order_pf
import asyncio
with IB().connect(port=PORT) as ib:
    x = asyncio.run(get_order_pf(PORT))